In [1]:
import os # Configure which GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import random
import torch

from utils.NN_utils import BeamPredictionModel, BlockPredictionModel, BestGainPredictionModel
from utils.NN_utils import BeamPredictionLSTMModel, BlockPredictionLSTMModel, BestGainPredictionLSTMModel
from utils.NN_utils import preprocess_data
from utils.options import args_parser
from utils.mox_utils import setup_seed, get_save_dirs, np2torch, save_NN_results
from utils.data_utils import get_prepared_dataset, prepare_dataset
from utils.beam_utils import generate_dft_codebook, beamIdPair_to_beamPairId, beamPairId_to_beamIdPair
from utils.NN_utils import BeamPredictionModel, train_beampred_lstm_model
from utils.options import args_parser
from utils.mox_utils import setup_seed, get_save_dirs, save_NN_results
from utils.plot_utils import plot_record_metrics
from utils.data_utils import get_prepared_dataset, prepare_dataset, augment_dataset
from utils.beam_utils import generate_dft_codebook


In [2]:
setup_seed(20)
N_bs = 4
freq = 28e9
DS_start, DS_end = 200, 800
preprocess_mode = 0
pos_in_data = preprocess_mode==2
look_ahead_len = 10
M_t = 32
M_r = 8
n_pilot = 32
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
lbd = 1
gpu = 5
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
        = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, 
                               look_ahead_len=look_ahead_len, lbd=lbd)    

data_torch, best_beam_pair_index_label, lengths = augment_dataset(
    data_np, best_beam_pair_index_np, look_ahead_len, augment_dataset_ratio=2)

Using device:  cuda:5


In [3]:
num_epochs =  50
data_size_test = 10000
data_torch_test = data_torch[:data_size_test,...]
lengths_test = lengths[:data_size_test,...]
best_beam_pair_index_label_test = best_beam_pair_index_label[:data_size_test,...]

record_metrics_list = []
best_model_weights_list = []

for Np in [1,2,4,8,16,32]:
    print(f'start training (Np={Np})')
    print(data_torch_test[:,:,::int(n_pilot/Np)].shape)
    best_model_weights, record_metrics = \
        train_beampred_lstm_model(num_epochs, device, data_torch_test[:,:,::int(n_pilot/Np)], lengths_test, best_beam_pair_index_label_test, M_t, M_r, pos_in_data=(preprocess_mode==2))
    record_metrics_list.append(record_metrics)
    best_model_weights_list.append(best_model_weights)


start training (Np=1)
torch.Size([10000, 10, 8])
Epoch 1/50, training time: 6.48s
loss: 2.9581/2.1969 | acc: 0.3919/0.4652 | top3_acc: 0.5579/0.6803
Epoch 2/50, training time: 1.33s
loss: 1.9069/1.8912 | acc: 0.5062/0.4985 | top3_acc: 0.7238/0.7259
Epoch 3/50, training time: 1.43s
loss: 1.6356/1.7851 | acc: 0.5387/0.5097 | top3_acc: 0.7736/0.7504
Epoch 4/50, training time: 1.35s
loss: 1.4106/1.6412 | acc: 0.5801/0.5435 | top3_acc: 0.8192/0.7764
Epoch 5/50, training time: 1.38s
loss: 1.2522/1.5393 | acc: 0.6119/0.5689 | top3_acc: 0.8531/0.7907
Epoch 6/50, training time: 1.43s
loss: 1.0964/1.5459 | acc: 0.6507/0.5656 | top3_acc: 0.8831/0.8012
Epoch 7/50, training time: 1.44s
loss: 0.9923/1.4795 | acc: 0.6782/0.5792 | top3_acc: 0.9018/0.8173
Epoch 8/50, training time: 1.41s
loss: 0.8830/1.4455 | acc: 0.7088/0.5973 | top3_acc: 0.9234/0.8214
Epoch 9/50, training time: 1.41s
loss: 0.7735/1.3870 | acc: 0.7451/0.6065 | top3_acc: 0.9416/0.8338
Epoch 10/50, training time: 1.42s
loss: 0.6956/1.40

In [6]:
def get_recored_bestKPIs(record_metrics_list, Np_list=[1,2,4,8,16,32]):
    epochs = len(record_metrics_list[0]['train_loss'])
    bestKPIs = np.zeros((len(Np_list), 6)) # Np, best_epoch, best_train_acc, best_val_acc, best_train_top3_acc, best_val_top3_acc
    for i, record_metrics in enumerate(record_metrics_list):
        best_epoch = np.argmax(record_metrics['val_acc'])
        bestKPIs[i,0] = Np_list[i]
        bestKPIs[i,1] = best_epoch+1
        bestKPIs[i,2] = record_metrics['train_acc'][best_epoch]
        bestKPIs[i,3] = record_metrics['val_acc'][best_epoch]
        bestKPIs[i,4] = record_metrics['train_top3_acc'][best_epoch]
        bestKPIs[i,5] = record_metrics['val_top3_acc'][best_epoch]
    return bestKPIs

print('Best KPIs (Np, best_epoch, best_train_acc, best_val_acc, best_train_top3_acc, best_val_top3_acc):')
get_recored_bestKPIs(record_metrics_list)

Best KPIs (Np, best_epoch, best_train_acc, best_val_acc, best_train_top3_acc, best_val_top3_acc):


array([[ 1.        , 48.        ,  0.94039286,  0.72083333,  0.9975    ,
         0.88075   ],
       [ 2.        , 47.        ,  0.96725   ,  0.80058333,  0.99928571,
         0.93108333],
       [ 4.        , 47.        ,  0.98007143,  0.83141667,  0.99946429,
         0.94083333],
       [ 8.        , 45.        ,  0.98367857,  0.83825   ,  0.99967857,
         0.94966667],
       [16.        , 50.        ,  0.98582143,  0.84783333,  0.99982143,
         0.95341667],
       [32.        , 43.        ,  0.97935714,  0.84625   ,  0.99964286,
         0.95241667]])

In [ ]:
data_torch = np2torch(data_np[:,0,...],device) 
veh_h_torch = np2torch(veh_h_np[:,-1,...],device) 
veh_pos_torch = np2torch(veh_pos_np[:,-1,...],device) 
best_beam_pair_index_torch = np2torch(best_beam_pair_index_np[:,-1,...],device)
block_labels = (veh_h_torch[:,0,:,0]==0).long().to(device)
DFT_tx = generate_dft_codebook(M_t)
DFT_rx = generate_dft_codebook(M_r)
beamPairId = best_beam_pair_index_torch.detach().cpu().numpy()
beamIdPair = beamPairId_to_beamIdPair(beamPairId,M_t,M_r)
num_car, _, num_bs, _ = veh_h_torch.shape
channel = veh_h_torch.detach().cpu().numpy()
g_opt = np.zeros((num_car,num_bs)).astype(np.float32)
for veh in range(num_car):
    for bs in range(num_bs):
        g_opt[veh,bs] = 1/np.sqrt(M_t*M_r)*np.abs(np.matmul(np.matmul(channel[veh,:,bs,:], DFT_tx[:,beamIdPair[veh,bs,0]]).T.conjugate(),DFT_rx[:,beamIdPair[veh,bs,1]]))
        g_opt[veh,bs] = 20 * np.log10(g_opt[veh,bs] + 1e-9) 
g_opt_normalized = g_opt / 20 + 7
gain_labels = torch.tensor(g_opt_normalized).to(device)

feature_input_dim = data_torch.shape[-1] * 2 - pos_in_data * 2
num_bs = N_bs
num_beampair = M_t * M_r

beam_pred_model = BeamPredictionLSTMModel(feature_input_dim, num_bs, num_beampair).to(device)
gain_pred_model = BestGainPredictionLSTMModel(feature_input_dim, num_bs).to(device)

beam_pred_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/beampred_lstm_valAcc93.01%_2025-04-29_20:55:34.pth'))
gain_pred_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/gainpred_lstm_valMae2.06dB_2025-04-29_20:18:49.pth'))

data = preprocess_data(data_torch, pos_in_data)
beam_pred_model.eval()
gain_pred_model.eval()
with torch.no_grad():
    beam_pred = beam_pred_model(data).argmax(dim=-1)
    gain_pred = gain_pred_model.pred(data)


In [5]:
beam_pred_model

BeamPredictionModel(
  (shared_layers): Sequential(
    (0): VectorResBlock(
      (fc1): Linear(in_features=258, out_features=128, bias=False)
      (bn1): BatchNorm1d(258, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=128, out_features=128, bias=False)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Linear(in_features=258, out_features=128, bias=False)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ReLU()
    (2): VectorResBlock(
      (fc1): Linear(in_features=128, out_features=64, bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=64, out_features=64, bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [ ]:
(block_pred == block_labels).to(torch.float).mean()

In [ ]:
np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()

In [ ]:
gain_pred.max(), gain_pred.min(), g_opt.max(), g_opt.min() #  -69.123604~-148.2894

In [ ]:
# gain_pred and block_pred coordination
np.abs(((gain_pred*(1-block_pred) - 180.*block_pred).detach().cpu().numpy() - g_opt)).mean()

In [ ]:
# Clip gain_pred
gain_pred_clipLB = gain_pred*(gain_pred>=-180.) - 180.*(gain_pred<-180.)
np.abs((gain_pred_clipLB.detach().cpu().numpy() - g_opt)).mean()

# LSTM

In [2]:
setup_seed(20)
freq = 28e9
DS_start, DS_end = 800, 950
preprocess_mode = 2
pos_in_data = preprocess_mode==2
look_ahead_len = 10
n_pilot = 16
M_r, N_bs, M_t = 8, 4, 64
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
gpu = 3
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
    = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, look_ahead_len)
data_torch = np2torch(data_np[:,:-1,...],device) 
veh_h_torch = np2torch(veh_h_np[:,-1,...],device) 
veh_pos_torch = np2torch(veh_pos_np[:,-1,...],device) 
best_beam_pair_index_torch = np2torch(best_beam_pair_index_np[:,-1,...],device)
block_labels = (veh_h_torch[:,0,:,0]==0).long().to(device)
DFT_tx = generate_dft_codebook(M_t)
DFT_rx = generate_dft_codebook(M_r)
beamPairId = best_beam_pair_index_torch.detach().cpu().numpy()
beamIdPair = beamPairId_to_beamIdPair(beamPairId,M_t,M_r)
num_car, _, num_bs, _ = veh_h_torch.shape
channel = veh_h_torch.detach().cpu().numpy()
g_opt = np.zeros((num_car,num_bs)).astype(np.float32)
for veh in range(num_car):
    for bs in range(num_bs):
        g_opt[veh,bs] = 1/np.sqrt(M_t*M_r)*np.abs(np.matmul(np.matmul(channel[veh,:,bs,:], DFT_tx[:,beamIdPair[veh,bs,0]]).T.conjugate(),DFT_rx[:,beamIdPair[veh,bs,1]]))
        g_opt[veh,bs] = 20 * np.log10(g_opt[veh,bs] + 1e-9) 
g_opt_normalized = g_opt / 20 + 7
gain_labels = torch.tensor(g_opt_normalized).to(device)

feature_input_dim = data_torch.shape[-1] * 2 - pos_in_data * 2
num_bs = 4
num_beampair = M_t * M_r

beam_pred_lstm_model = BeamPredictionLSTMModel(feature_input_dim, num_bs, num_beampair).to(device)
block_pred_lstm_model = BlockPredictionLSTMModel(feature_input_dim, num_bs).to(device)
gain_pred_lstm_model = BestGainPredictionLSTMModel(feature_input_dim, num_bs).to(device)

beam_pred_lstm_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/beampred_lstm_valAcc93.01%_2025-04-29_20:55:34.pth'))
block_pred_lstm_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead3/models/blockpred_lstm_valAcc98.87%_2025-04-26_13:08:55.pth'))
gain_pred_lstm_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/gainpred_lstm_valMae2.06dB_2025-04-29_20:18:49.pth'))

data = preprocess_data(data_torch, pos_in_data)
beam_pred_lstm_model.eval()
block_pred_lstm_model.eval()
gain_pred_lstm_model.eval()

Using device:  cuda:3


BestGainPredictionLSTMModel(
  (lstm_layers): LSTM(258, 128, batch_first=True)
  (shared_layers): Sequential()
  (output_heads): ModuleList(
    (0-3): 4 x Sequential(
      (0): VectorResBlock(
        (fc1): Linear(in_features=128, out_features=128, bias=False)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fc2): Linear(in_features=128, out_features=128, bias=False)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): ReLU()
      (2): VectorResBlock(
        (fc1): Linear(in_features=128, out_features=128, bias=False)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fc2): Linear(in_features=128, out_features=128, bias=False)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (3): ReL

In [3]:
for input_len in range(1,look_ahead_len+1):
    input_data = data[:10000,-input_len:,:]
    with torch.no_grad():
        beam_pred = beam_pred_lstm_model(input_data).argmax(dim=-1)
        block_pred = block_pred_lstm_model(input_data).argmax(dim=-1)
        gain_pred = gain_pred_lstm_model.pred(input_data)

    acc_block_pred = (block_pred == block_labels).to(torch.float).mean()
    acc_beam_pred = (beam_pred == best_beam_pair_index_torch).to(torch.float).mean()
    mae_gain_pred = np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()
    print('Input length: ', input_len)
    print(f'--Block prediction accuracy: {acc_block_pred.item():.4f}', )
    print(f'--Beam prediction accuracy: {acc_beam_pred.item():.4f}', )
    print(f'--Mean absolute error of gain prediction: {mae_gain_pred.item():.4f}', )

RuntimeError: The size of tensor a (10000) must match the size of tensor b (146373) at non-singleton dimension 0

In [ ]:
for input_len in range(1,look_ahead_len+1):
    input_data = data[:,-input_len:,:]
    with torch.no_grad():
        beam_pred = beam_pred_lstm_model(input_data).argmax(dim=-1)
        block_pred = block_pred_lstm_model(input_data).argmax(dim=-1)
        gain_pred = gain_pred_lstm_model.pred(input_data)

    acc_block_pred = (block_pred == block_labels).to(torch.float).mean()
    acc_beam_pred = (beam_pred == best_beam_pair_index_torch).to(torch.float).mean()
    mae_gain_pred = np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()
    print('Input length: ', input_len)
    print(f'--Block prediction accuracy: {acc_block_pred.item():.4f}', )
    print(f'--Beam prediction accuracy: {acc_beam_pred.item():.4f}', )
    print(f'--Mean absolute error of gain prediction: {mae_gain_pred.item():.4f}', )

Input length:  1
--Block prediction accuracy: 0.9791
--Beam prediction accuracy: 0.8902
--Mean absolute error of gain prediction: 2.6942
Input length:  2
--Block prediction accuracy: 0.9814
--Beam prediction accuracy: 0.8959
--Mean absolute error of gain prediction: 2.4839
Input length:  3
--Block prediction accuracy: 0.9818
--Beam prediction accuracy: 0.8965
--Mean absolute error of gain prediction: 2.4575


In [ ]:
gain_pred.max(), gain_pred.min(), g_opt.max(), g_opt.min() #  -69.123604~-148.2894

(tensor(-66.5101, device='cuda:4'),
 tensor(-187.5919, device='cuda:4'),
 -64.07656,
 -180.0)

In [ ]:
# gain_pred and block_pred coordination
np.abs(((gain_pred*(1-block_pred) - 180.*block_pred).detach().cpu().numpy() - g_opt)).mean()

2.4202566

In [ ]:
# Clip gain_pred
gain_pred_clipLB = gain_pred*(gain_pred>=-180.) - 180.*(gain_pred<-180.)
np.abs((gain_pred_clipLB.detach().cpu().numpy() - g_opt)).mean()

2.4469101

In [ ]:
(gain_pred.detach().cpu().numpy()-g_opt).reshape(-1)

array([ 0.90330505,  0.11447144, -0.89450836, ...,  2.6871262 ,
       -0.29328918,  1.3418427 ], dtype=float32)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist((gain_pred.detach().cpu().numpy()-g_opt).reshape(-1), bins=100)
plt.show()

NameError: name 'gain_pred' is not defined

In [ ]:
block_pred.shape, gain_pred.shape

(torch.Size([147903, 4]), torch.Size([147903, 4]))

In [14]:
block_pred_lstm_model(data).shape, gain_pred_lstm_model.pred(data).shape

(torch.Size([147903, 4, 2]), torch.Size([147903, 4]))

In [15]:
output_block_pred_lstm_model = block_pred_lstm_model(data)
output_gain_pred_lstm_model = gain_pred_lstm_model(data)

In [16]:
output_block_pred_lstm_model.shape, output_block_pred_lstm_model

(torch.Size([147903, 4, 2]),
 tensor([[[  6.6171,  -4.2506],
          [ -5.9450,   6.9206],
          [  7.7003,  -8.2714],
          [  5.5485,  -7.2104]],
 
         [[ 12.5596, -11.4817],
          [ 12.8745, -10.7068],
          [  6.1245,  -6.6015],
          [  1.3027,  -1.2121]],
 
         [[ 15.5200, -13.9335],
          [ 10.5807,  -7.5470],
          [  2.1299,  -2.1151],
          [-10.5589,  10.4850]],
 
         ...,
 
         [[ -4.5845,   4.5399],
          [  7.5996,  -6.2444],
          [  7.9857,  -8.7114],
          [  5.6381,  -6.6196]],
 
         [[ -5.2027,   5.1565],
          [  3.5624,  -3.5991],
          [ 10.5413, -11.7617],
          [  7.2885,  -8.7141]],
 
         [[ -6.5844,   6.5532],
          [  2.5709,  -2.5035],
          [ 11.3501, -12.4060],
          [  7.5030,  -9.0474]]], device='cuda:4', grad_fn=<StackBackward0>))

In [17]:
output_gain_pred_lstm_model.shape, output_gain_pred_lstm_model

(torch.Size([147903, 4]),
 tensor([[ 2.4703, -1.9943,  3.1746,  2.6520],
         [ 3.0827,  2.6464,  2.6034,  0.3447],
         [ 2.8247,  2.3920,  0.5945, -2.0076],
         ...,
         [-1.5296,  2.0523,  2.5126,  3.2083],
         [-1.9940,  1.7486,  2.3294,  3.1009],
         [-2.0009,  1.2814,  2.3381,  3.0026]], device='cuda:4',
        grad_fn=<CatBackward0>))

In [18]:
torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1).shape, torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1)

(torch.Size([147903, 4]),
 tensor([[0, 1, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 1],
         ...,
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0]], device='cuda:4'))

In [19]:
error_loc = ~(block_labels == torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1))